# Objectif de la tâche:
Le but de cet exercice est d'utiliser un réseau de neurones réccurrent pour prédire (ou compléter) le mot manquant dans un proverbe. Le RNN utilisé dans cette tâche est le LSTM (Long Short Term Memory). 
Pour ce faire, nous avons défini quelques fonctions utilitaires, le modèle LSTM qui sera entrainé puis évalué.

#### load dataset and some utils functions

Pour chaque proverbe, les plongements français de Spacy sont utilisés pour avoir son ***embedding*** correspondant que sera utilisé dans le LSTM.
La fonction **get_vocabulary** est définie pour l'obtenetion d'un vocabulaire fermé i.e. un vocabulaire constitué unique des mots des documents de ***train*** et ***test***. Cette définition de vocabulaire fermé permet à notre modèle de connaître tous les mots dans le but d'éviter le problème des mots inconnus. La fonction ***load_dataset*** permet de constituer les corpus du train et du test. Ces derniers sont combinés pour former le vocabulaire fermé.

In [ ]:
import spacy

spacy_fr = spacy.load('fr_core_news_lg')
spacy_embedding_dim = spacy_fr.meta['vectors']['width']

In [ ]:
def get_vocabulary(corpus, spacy_analyzer, start_of_string=True, start_of_string_id=0, start_of_string_token='/<BOS>/', end_of_string=True, end_of_string_id=1, end_of_string_token='/<EOS>/'):
    token_set = set()
    
    for sentence in corpus:
        doc = spacy_analyzer(sentence)
        for token in doc:
            token_set.add(token.text)
    
    vocabulary = list(token_set)

    if start_of_string:
        vocabulary.insert(start_of_string_id, start_of_string_token)

    if end_of_string:
        vocabulary.insert(end_of_string_id, end_of_string_token)

    return vocabulary

In [ ]:
import json

def load_dataset(path, format_type='txt'):    
    with open(path, encoding='utf-8') as f:
        if format_type == 'json':
            return json.load(f)

        return [line.rstrip() for line in f]

In [ ]:
full_train_dataset = load_dataset('data/proverbes.txt')
test_dataset = load_dataset('data/test_proverbes.txt', format_type='json')
len(full_train_dataset), len(test_dataset)

In [ ]:
# Vocabulary is closed so we get the words of test dataset
test_words = []
for sentence_to_complete, propositions in test_dataset.items():
    test_words.append(sentence_to_complete.replace(" ***", ''))
    test_words.append(' '.join(propositions))

In [ ]:
vocab = get_vocabulary(full_train_dataset + test_words, spacy_fr)
print(len(vocab))

In [ ]:
index_to_word = {index: word for index, word in enumerate(vocab)}
word_to_index = {word: index for index, word in enumerate(vocab)}

***ProverbDataset*** est une classe qui hérite de ***Dataset*** de Pytorch. Cette classe est utilisée pour la préparation des doonées qui seront utilisées par l'entrainement, la validation et l'évaluation du modèle.
Le principe ***ground truth*** est utilisé dans la fonction ***tokenize***. Celui constitue à définir, pour chaque mot, le mot suivant comme sa target. Cela permet d'entrainer le modèle avec la stratégie de type ***teacher forcing***. Pour chaque proverbe, des délimiteurs de phrase sont utilisés pour encadrer les tokens de celle-ci. Les délimiteurs ***<BOS>*** et ***<EOS>*** sont utilisés pour matérialiser le début et la fin de la phrase respectivement. La fonction ***fill_dataset_with_targets*** donne toutes les données et targets d'un dataset donné.

In [ ]:
import torch
from torch import LongTensor, FloatTensor
from torch.utils.data import DataLoader, Dataset

class ProverbDataset(Dataset):
    def __init__(self, dataset, word_to_index, spacy_analyzer):
        self.dataset = dataset
        self.tokenizer = spacy_analyzer
        self.word_to_index = word_to_index
        # Construct dataset such as the next word is a target of the previous
        self.dataset_with_targets = self.fill_dataset_with_targets() # [w(0), w(1), ...w(n-1)],[w(1), w(2), ...w(n)] 

    def __len__(self):
        return len(self.dataset_with_targets[0])

    def __getitem__(self, index):
        return LongTensor([self.dataset_with_targets[0][index]]), FloatTensor([self.dataset_with_targets[1][index]]).squeeze(0)

    def tokenize(self, sentence):
        tokens = [word.text for word in self.tokenizer(sentence)]
        tokens.insert(0, '/<BOS>/')
        tokens.append('/<EOS>/')

        data = []
        targets = []
        for index in range(len(tokens)-1):
            data.append(self.word_to_index.get(tokens[index], 1))
            targets.append(self.word_to_index.get(tokens[index+1], 1))

        return data, targets

    def fill_dataset_with_targets(self):
        full_data = []
        full_targets = []
        for sentence in self.dataset:
            data, targets = self.tokenize(sentence)
            full_data.extend(data)
            full_targets.extend(targets)

        assert len(full_data) == len(full_targets)
        return full_data, full_targets

**Division du jeu de données d'entrainement en des sous-ensembles de train et de validation**
Cette approche permet d'entrainer puis de valider notre modèle avant de l'évaluer avec les données du test.

In [ ]:
valid_ratio = 0.1
valid_size = int(len(full_train_dataset) * valid_ratio)
train_size = len(full_train_dataset) - valid_size

X_train = full_train_dataset[:train_size]
X_valid = full_train_dataset[train_size:]

**Définition des différentes variables et constantes qui seront utilisées pour la définition du modèle LSTM**

In [ ]:
import numpy as np

# Get spacy embeddings of all the words in vocabulary
vocab_size = len(vocab)
embedding_size = spacy_embedding_dim
embedding_layer = np.zeros((vocab_size, embedding_size), dtype=np.float32)
for index, word in index_to_word.items():
    embedding_layer[index, :] = spacy_fr(word).vector

embedding_layer = torch.from_numpy(embedding_layer)
embedding_layer.shape

#### LSTM

Le modèle LSTM est défini par les vecteurs de plongements définis précédemment, ***embedding_layer***. Ceux-ci sont utilisés pour définir la taille des vecteurs de plongements et la couche de plongement du LSTM. Ensuite, les dimensions des vecteurs des couches cachée et de sortie sont initialisées, ***hidden_dim et output_dim***. Pour la récurrence du modèle, un autre LSTM est défini dans le modèle. Pour connecter les couches cachée et de sortie, la fonction linéaire est utilisée.
Pour l'entrainement, ***forward***, l'input est est passé au travers de la couche de plongements, ***embedding_layer*** pour définir son vecteur de plongement. Le vecteur obtenu est passé à la couche ***lstm_layer*** pour donner la prédiction et le couple hidden et context qui sera utilisé dans la constitution des prochains inputs. Le vecteur de prédiction résultant est compressé puis passe au travers de la couche linéaire, ***fc_layer*** pour déterminer la probabilité de sortie de l'input.

Pour entrainer notre modèle, on constitue des séquences de deux mots [$w_{i-1}$ $wi$] où $wi$ est considéré comme la target de $w{i-1}$. 
Pour chaque phrase dans le dataset, on découpe la phrase en bigrammes et on renvoie pour l'index correspondant les tensors correspondants qui seront ensuite chargés en minibatch grâce au Dataloader. 

Pour chaque minibatch tiré de notre loader, on le passe dans notre modèle lstm constitué d'une couche d'embeddings préentrainée de spacy (len(vocabulary)xspacy_embedding_dim), une couche lstm (spacy_embedding_dim x hidden_dim) et d'une couche pleinement connectée (hidden_dim, output_dim).

In [ ]:
from torch import nn

class ModelLanguageLSTM(nn.Module):
    def __init__(self, embeddings, hidden_dim, output_dim):
        super(ModelLanguageLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_layer = nn.Embedding.from_pretrained(embeddings)
        self.embedding_size = embeddings.size()[1]
        self.lstm_layer = nn.LSTM(self.embedding_size, hidden_dim, 1, batch_first=True, bidirectional=False)
        self.fc_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding_layer(x)
        x, (h, ctx) = self.lstm_layer(x)
        x = h.squeeze()
        x = self.fc_layer(x)
        return x

#### Training with poutyne

In [ ]:
hidden_dim = 4
output_dim = 1

lstm = ModelLanguageLSTM(embedding_layer, hidden_dim, output_dim)

In [ ]:
train_dataset = ProverbDataset(X_train, word_to_index, spacy_fr)
valid_dataset = ProverbDataset(X_valid, word_to_index, spacy_fr)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True)

In [45]:
import shutil

try:
    shutil.rmtree('results/Q2')
except:
    print('Ignored!')

In [46]:
def loss_fn(y_pred, y_target):
    y_pred = torch.nn.functional.softmax(y_pred.squeeze(1))
    return nn.functional.cross_entropy(y_pred, y_target)

In [47]:
from poutyne import set_seeds
from poutyne.framework import Experiment

set_seeds(42)

experiment = Experiment("results/Q2/", lstm, loss_function=loss_fn, batch_metrics=["acc"], optimizer="SGD")
experiment.train(train_dataloader, valid_dataloader, epochs=25, disable_tensorboard=True)

Epoch:  1/25 Step:   59/2079   2.84% |▌                   |ETA: 5.47s loss: 89788.851562 acc: 0.000000 

C:\Users\wansijou\AppData\Local\Temp\ipykernel_25728\921777836.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_pred = torch.nn.functional.softmax(y_pred.squeeze(1))


Epoch:  1/25 Train steps: 2079 Val steps: 218 4.39s loss: 90594.325867 acc: 0.000000 val_loss: 88277.338753 val_acc: 0.000000
Epoch 1: val_loss improved from inf to 88277.33875, saving file to results/Q2/checkpoint_epoch_1.ckpt
Epoch:  2/25 Train steps: 2079 Val steps: 218 3.67s loss: 90592.600836 acc: 0.000000 val_loss: 88208.808919 val_acc: 0.000000
Epoch 2: val_loss improved from 88277.33875 to 88208.80892, saving file to results/Q2/checkpoint_epoch_2.ckpt
Epoch:  3/25 Train steps: 2079 Val steps: 218 3.75s loss: 90603.294334 acc: 0.000000 val_loss: 88289.188115 val_acc: 0.000000
Epoch:  4/25 Train steps: 2079 Val steps: 218 3.92s loss: 90593.612790 acc: 0.000000 val_loss: 88232.159515 val_acc: 0.000000
Epoch:  5/25 Train steps: 2079 Val steps: 218 3.71s loss: 90611.902362 acc: 0.000000 val_loss: 88306.494701 val_acc: 0.000000
Epoch:  6/25 Train steps: 2079 Val steps: 218 3.80s loss: 90603.887892 acc: 0.000000 val_loss: 88260.198904 val_acc: 0.000000
Epoch:  7/25 Train steps: 2079 V

[{'epoch': 1,
  'time': 4.394115600000077,
  'loss': 90594.32586692131,
  'acc': 0.0,
  'val_loss': 88277.3387531241,
  'val_acc': 0.0},
 {'epoch': 2,
  'time': 3.6748869999998988,
  'loss': 90592.60083612052,
  'acc': 0.0,
  'val_loss': 88208.80891852165,
  'val_acc': 0.0},
 {'epoch': 3,
  'time': 3.7532951999999113,
  'loss': 90603.29433428704,
  'acc': 0.0,
  'val_loss': 88289.1881147062,
  'val_acc': 0.0},
 {'epoch': 4,
  'time': 3.9187580999998772,
  'loss': 90593.61279048718,
  'acc': 0.0,
  'val_loss': 88232.15951457314,
  'val_acc': 0.0},
 {'epoch': 5,
  'time': 3.714496299999837,
  'loss': 90611.90236204605,
  'acc': 0.0,
  'val_loss': 88306.4947013899,
  'val_acc': 0.0},
 {'epoch': 6,
  'time': 3.7970066999998835,
  'loss': 90603.88789181145,
  'acc': 0.0,
  'val_loss': 88260.19890353046,
  'val_acc': 0.0},
 {'epoch': 7,
  'time': 3.7358277999999245,
  'loss': 90606.76477856556,
  'acc': 0.0,
  'val_loss': 88171.55325580768,
  'val_acc': 0.0},
 {'epoch': 8,
  'time': 3.788734

#### Test and evaluate lstm model

Le modèle entrainé est utilisé pour la prédiction du mot manquant. La démarche adoptée est la suivante:
- Pour chaque proverbe de test, le mot manquant est remplacé par une des propositions
- Le proverbe "complet" obtenu est tokenisé
- Les tokens sont ensuite convertis en une liste de leurs index respectifs. Celle est transformée en tensor avant d'être utilisé par le modèle pour donner la probabilité. 
- La proposition du proverbe avec la plus grande probabilité est sélectionnée comme le mot manqaunt idéal

In [48]:
def get_prediction(sentence_to_complete, propositions, target_token="***"):
    probs = []
    possible_proverbs = [sentence_to_complete.replace(target_token, w) for w in propositions]
    for p in possible_proverbs:
        tokens = [word.text for word in spacy_fr(p)]
        tokens.insert(0, '/<BOS>/')
        tokens.append('/<EOS>/')

        data = []
        for idx in range(len(tokens)-1):
            data.append(word_to_index[tokens[idx]])

        out = lstm(LongTensor(data))

    to_predict = np.argmax(np.average(out.detach().numpy()))
    return propositions[to_predict]

In [49]:
predictions = []

# Get all predictions of our test dataset
for sentence_to_complete, propositions in test_dataset.items():
    pred = get_prediction(sentence_to_complete, propositions)
    predictions.append(pred)
    print(f"{sentence_to_complete} ----> {pred}")

a beau mentir qui *** de loin ----> vient
a beau *** qui vient de loin ----> mentir
l’occasion fait le *** ----> larron
aide-toi, le ciel t’*** ----> aidera
année de gelée, *** de blé ----> année
après la pluie, le *** temps ----> beau
aux échecs, les *** sont les plus près des rois ----> fous
ce que *** veut, dieu le veut ----> femme
bien mal acquis ne *** jamais ----> profite
bon ouvrier ne querelle pas ses *** ----> outils
ce n’est pas tous les jours *** ----> fête
pour le fou, c’est tous les jours *** ----> fête
dire et faire, *** deux ----> sont
mieux vaut *** que jamais ----> tard
d’un sac *** ne peut tirer deux moutures ----> on
à qui dieu aide, *** ne peut nuire ----> nul
il n’y a *** de rose de cent jours ----> point
il faut le *** pour le croire ----> voir
on ne *** pas le poisson qui est encore dans la mer ----> vend
la langue d’un *** vaut mieux que celle d’un menteur ----> muet
*** femme fait le bon homme ----> bonne
bonne *** fait le bon homme ----> femme
bonne femme *** 

In [50]:
# We've created a file with the solution of the sentence in test dataset
solutions = load_dataset("data/solutions.txt")

In [51]:
def precision_metric(y_true, y_pred):
    true_pred = 0

    for idx in range(len(y_pred)):
        if y_true[idx] == y_pred[idx]:
            true_pred += 1
    
    return true_pred / len(y_pred)

In [52]:
precision_metric(solutions, predictions)

0.9347826086956522

En conclusion, force est de constater que le LSTM créé est très instable. Après plusieurs tests, nous avons réussi à avoir une précision de 93%. Cependant, cette très bonne valeur de la précision du modèle nous laisse perplexe sur la robustesse du fait l'instabilité du modèle. En effet, les différentes précisions obtenues au cours de l'expérimentation varient entre 20 et 93%. Cette instabilité pourrait être expliquée par l'approche ground truth. Avec cette stratégie, un mot peut se retrouver à avoir plusieurs targets possibles. De ce fait, le modèle fera une sélection aléatoire. En perspective, il serait intéressant de faire un LSTM bidirectionnel pour avoir beaucoup plus de contexte afin d'aider le modèle à prendre la meilleure décision. 

Par comparaison au modèle du TP1 qui utilisait l'algorithme Laplace, le LSTM semble être beaucoup moins robuste. Les résultats du LSTM ressemblent à ceux du modèle Laplace en unigramme. Cela nous pousse à nous poser la question sur notre algorithme de ground truth. Il serait intéressant alors d'utiliser un algorithme dans lequel les données et targets sont constituées graduellement 